# Traitement et classification des fichiers GPKG

**Objectif:** Téléchargement, vérification, transformation et visualisation de fichiers GPKG (Geopackage) 
pour la classification automatique des données de toitures.

**Workflow principal:**
1. Chargement et validation des données GPKG
2. Classification automatique des toitures selon critères définis (pas retenu pour la méthodologie)
3. Visualisation et export des résultats

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
from pathlib import Path
import re

from tqdm.notebook import tqdm
from IPython.display import display, SVG

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import seaborn as sns

from graphviz import Digraph
from cairosvg import svg2png

import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.errors import ShapelyDeprecationWarning
from shapely.prepared import prep

import warnings

from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

## Variables

In [ ]:
todaysdate = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
NOTEBOOK2_PATH = "data/notebook_02"
OUTPUT_GRAPHICS_NOTEBOOK_02_PATH = Path(NOTEBOOK2_PATH) / "graphics"
OUTPUT_PARQUET_NOTEBOOK_02_PATH = Path(NOTEBOOK2_PATH) / "parquet"

SITG_GPKG_PATH = "data/SITG"
CAD_BATIMENT_HORSOL_GPKG_PATH = Path(SITG_GPKG_PATH) / "CAD_BATIMENT_HORSOL_2024-11-03.gpkg"
CAD_BATIMENT_HORSOL_TOIT_GPKG_PATH = Path(SITG_GPKG_PATH) / "CAD_BATIMENT_HORSOL_TOIT_2024-11-03.gpkg"
CAD_BATIMENT_HORSOL_TOIT_SP_GPKG_PATH = Path(SITG_GPKG_PATH) / "CAD_BATIMENT_HORSOL_TOIT_SP_2024-11-03.gpkg"
CAD_COMMUNE_GPKG_PATH = Path(SITG_GPKG_PATH) / "CAD_COMMUNE_2024-11-03.gpkg"


In [ ]:
os.makedirs(OUTPUT_GRAPHICS_NOTEBOOK_02_PATH, exist_ok=True)
os.makedirs(OUTPUT_PARQUET_NOTEBOOK_02_PATH, exist_ok=True)
os.makedirs(SITG_GPKG_PATH, exist_ok=True)

assert CAD_BATIMENT_HORSOL_GPKG_PATH.exists(), f"File {CAD_BATIMENT_HORSOL_GPKG_PATH} does not exist."
assert CAD_BATIMENT_HORSOL_TOIT_GPKG_PATH.exists(), f"File {CAD_BATIMENT_HORSOL_TOIT_GPKG_PATH} does not exist."
assert CAD_BATIMENT_HORSOL_TOIT_SP_GPKG_PATH.exists(), f"File {CAD_BATIMENT_HORSOL_TOIT_SP_GPKG_PATH} does not exist."
assert CAD_COMMUNE_GPKG_PATH.exists(), f"File {CAD_COMMUNE_GPKG_PATH} does not exist."

## Helpers

### SVG

In [ ]:
def save_and_display_svg(svg_content, file_path, scale_factor=8):
    """
    Sauvegarde et affiche un contenu SVG avec conversion en PNG haute qualité.
    
    Parameters:
        svg_content (str): Contenu SVG à sauvegarder
        file_path (str): Chemin de sauvegarde du fichier
        scale_factor (int): Facteur de mise à l'échelle pour la qualité PNG
    """
    # Sauvegarde du SVG
    with open(file_path, "w") as f:
        f.write(svg_content)
    display(SVG(file_path))

    # Extraction des dimensions depuis le viewBox
    viewbox_match = re.search(r'viewBox="([\d\s]+)"', svg_content)
    if viewbox_match:
        viewbox = viewbox_match.group(1).split()
        width = int(float(viewbox[2]))
        height = int(float(viewbox[3]))
    else:
        width = 800
        height = 580

    # Calcul des dimensions pour PNG haute qualité
    png_width = width * scale_factor
    png_height = height * scale_factor

    # Conversion en PNG
    svg2png(
        url=file_path,
        write_to=file_path.replace(".svg", ".png"),
        background_color="white",
        scale=scale_factor,
        output_width=png_width,
        output_height=png_height,
        unsafe=False,
    )

    print(f"PNG sauvegardé: {png_width}x{png_height} pixels")

### Gestion des graphiques Graphviz

In [ ]:
def save_high_quality_graph(graph, filename, scale_factor=4):
    """
    Génère un PNG haute qualité à partir d'un graphique Graphviz.
    
    Parameters:
        graph: Objet Graphviz
        filename (str): Nom du fichier de sortie (sans extension)
        scale_factor (int): Facteur de qualité
    """
    # Rendu initial en SVG
    graph.render(filename, format="svg", cleanup=True)

    # Lecture du SVG généré
    with open(f"{filename}.svg", "r") as f:
        svg_content = f.read()

    # Extraction des dimensions
    width_match = re.search(r'width="(\d+)pt"', svg_content)
    height_match = re.search(r'height="(\d+)pt"', svg_content)

    if width_match and height_match:
        # Conversion pt vers px avec facteur de qualité
        width = int(float(width_match.group(1)) * 1.33 * scale_factor)
        height = int(float(height_match.group(1)) * 1.33 * scale_factor)
    else:
        width = 1000 * scale_factor
        height = 1000 * scale_factor

    # Conversion en PNG haute qualité
    svg2png(
        url=f"{filename}.svg",
        write_to=f"{filename}.png",
        background_color="white",
        scale=scale_factor,
        output_width=width,
        output_height=height,
    )

    # Suppression du fichier SVG après conversion
    os.remove(f"{filename}.svg")

    print(f"PNG haute qualité sauvegardé ({width}x{height} pixels)")

## Graphviz notebook

In [ ]:
def create_optimized_data_flow_diagram():
    """Crée le diagramme de flux ETL pour le traitement des fichiers GPKG."""
    g = Digraph("G", format="png")
    g.attr(
        rankdir="TB",
        bgcolor="transparent",
        fontname="Arial",
        pad="1.0",
        nodesep="1.0",
        ranksep="0.8",
        ratio="compress",
    )

    # Configuration des styles de nœuds
    g.attr(
        "node",
        shape="box",
        style="rounded,filled",
        fontname="Arial",
        fontsize="11",
        margin="0.2",
        width="2.0",
        height="0.5",
    )

    # Définition des styles
    file_style = {"fillcolor": "#E3F2FD", "color": "#1565C0", "height": "0.4"}
    output_file_style = {
        "fillcolor": "#E3F2FD",
        "color": "#1565C0",
        "height": "0.4",
        "penwidth": "2.5",
        "style": "rounded,filled,bold",
    }
    process_style = {"fillcolor": "#F1F8E9", "color": "#558B2F", "height": "0.4"}
    validation_style = {"fillcolor": "#FFF3E0", "color": "#EF6C00", "shape": "diamond"}
    action_style = {"fillcolor": "#E8EAF6", "color": "#3949AB", "height": "0.4"}

    # Création du cluster principal
    with g.subgraph(name="cluster_0") as c:
        c.attr(
            label="1. Téléchargement, Chargement, vérification et transformation des données des fichiers GPKG",
            style="rounded",
            color="#2A4D7E",
            margin="20",
        )

        # Flux initial
        with c.subgraph() as s:
            s.attr(rank="source")
            s.node(
                "gpkg_files",
                "Fichiers GPKG de SITG\n\nCAD_COMMUNE\nCAD_BATIMENT_HORSOL_TOIT\nCAD_BATIMENT_HORSOL_TOIT_SP\nCAD_BATIMENT_HORSOL",
                **file_style
            )
            s.node("validate_geo", "Validation\ngéométries", **validation_style)
            s.node("check_crs", "Vérification CRS\n(EPSG:2056)", **validation_style)
            s.node("check_dtypes", "Vérification et\nvalidation\ndtypes", **validation_style)

        # Étapes séquentielles
        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("celigny", "Bug REST API\navec Céligny", **validation_style)
            s.node("action_qgis", "Action: Téléchargement\nmanuel avec QGIS", **action_style)
            s.node("batiments_hc", "Toitures\nhors Canton", **validation_style)
            s.node("action_remove_hc", "Action: Retirer\npolygones\nhors canton", **action_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("gpkg_files_01", "02_gdf_toiture_1_filtre_canton.gpkg", **file_style)
            s.node(
                "gpkg_files_sp_01",
                "02_gdf_toiture_sp_1_filtre_canton.gpkg",
                **file_style
            )
            s.node(
                "egid",
                "EGID toiture\nnon associé avec\nEGID d'un bâtiment",
                **validation_style
            )
            s.node(
                "action_egid",
                "Action: Filtrer\nEGID toiture/toiture_sp\nnon associés avec\nEGID d'un bâtiment",
                **action_style
            )

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node(
                "gpkg_files_02",
                "02_gdf_toiture_2_filtre_egid_elimines.gpkg",
                **file_style
            )
            s.node(
                "gpkg_files_sp_02",
                "02_gdf_toiture_sp_2_filtre_egid.gpkg",
                **output_file_style
            )
        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("sia_cat", "Ajout Catégories\nSIA", **process_style)
            s.node(
                "gpkg_files_03",
                "02_gdf_toiture_3_ajout_cat_sia.gpkg",
                **output_file_style
            )

    # Ajout des connexions
    g.edge("gpkg_files", "validate_geo")
    g.edge("validate_geo", "check_crs")
    g.edge("check_crs", "check_dtypes")
    g.edge("check_dtypes", "celigny")
    g.edge("celigny", "action_qgis")
    g.edge("action_qgis", "batiments_hc")
    g.edge("batiments_hc", "action_remove_hc")
    g.edge(
        "action_remove_hc",
        "gpkg_files_sp_01",
        constraint="true",
        tailport="e",
        headport="n",
    )
    g.edge("action_remove_hc", "gpkg_files_01", constraint="true", tailport="sw", headport="n")
    g.edge("gpkg_files_01", "egid", constraint="true", tailport="s", headport="sw")
    g.edge("gpkg_files_sp_01", "egid")
    g.edge("egid", "action_egid")
    g.edge("action_egid", "gpkg_files_02")
    g.edge("action_egid", "gpkg_files_sp_02")
    g.edge("gpkg_files_02", "sia_cat")
    g.edge("sia_cat", "gpkg_files_03")

    return g

# Génération et sauvegarde du diagramme
g = create_optimized_data_flow_diagram()
save_high_quality_graph(
    g, f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_01_ETL_GPKG", scale_factor=4
)
g

In [ ]:
def create_optimized_data_flow_diagram():
    """Crée le diagramme de flux pour la classification des toitures."""
    g = Digraph("G", format="png")
    g.attr(
        rankdir="TB",
        bgcolor="transparent",
        fontname="Arial",
        pad="1.0",
        nodesep="1.0",
        ranksep="0.8",
        ratio="compress",
    )

    # Configuration des styles
    g.attr(
        "node",
        shape="box",
        style="rounded,filled",
        fontname="Arial",
        fontsize="11",
        margin="0.2",
        width="2.0",
        height="0.5",
    )

    # Définition des styles
    file_style = {"fillcolor": "#E3F2FD", "color": "#1565C0", "height": "0.4"}
    output_file_style = {
        "fillcolor": "#E3F2FD",
        "color": "#1565C0",
        "height": "0.4",
        "penwidth": "2.5",
        "style": "rounded,filled,bold",
    }
    process_style = {"fillcolor": "#F1F8E9", "color": "#558B2F", "height": "0.4"}
    validation_style = {"fillcolor": "#FFF3E0", "color": "#EF6C00", "shape": "diamond"}
    action_style = {"fillcolor": "#E8EAF6", "color": "#3949AB", "height": "0.4"}

    # Création du cluster de classification
    with g.subgraph(name="cluster_1") as c:
        c.attr(
            label="2. Classification des Toitures",
            style="rounded",
            color="#2A4D7E",
            margin="20",
        )

        # Organisation des nœuds par niveaux
        with c.subgraph() as s:
            s.attr(rank="source")
            s.node(
                "gpkg_files_sp_02", "02_gdf_toiture_sp_2_filtre_egid.gpkg", **file_style
            )
            s.node("gpkg_files_03", "02_gdf_toiture_3_ajout_cat_sia.gpkg", **file_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("classify_start", "Classification\nToitures", **process_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("surface_check", "Surface > 2m²?", **validation_style)
            s.node("gpkg_files_04", "02_gdf_toiture_4_polgone_2m2.gpkg", **file_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("classe1", "Classe 1:\nTotalement occupée", **action_style)
            s.node(
                "trou_check",
                "Polygone toiture avec\ntrou contenu dedans?",
                **validation_style
            )

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("gpkg_files_05", "02_gdf_toiture_5_hole.gpkg", **file_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("sp_check", "Superstructure?", **validation_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("gpkg_files_06", "02_gdf_toiture_6_contains_SP.gpkg", **file_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("classe2", "Classe 2a/2b:\nPartiellement occupée", **action_style)
            s.node("classe3", "Classe 3a/3b:\nLibre", **action_style)

        with c.subgraph() as s:
            s.attr(rank="sink")
            s.node(
                "gpkg_files_07",
                "02_gdf_toiture_7_classification.gpkg",
                **output_file_style
            )

    # Ajout des connexions
    g.edge("gpkg_files_sp_02", "classify_start")
    g.edge("gpkg_files_03", "classify_start")
    g.edge("classify_start", "surface_check")
    g.edge("surface_check", "gpkg_files_04")
    g.edge("gpkg_files_04", "classe1", "≤ 2m²")
    g.edge("gpkg_files_04", "trou_check", "> 2m²")
    g.edge("trou_check", "gpkg_files_05")
    g.edge("gpkg_files_05", "sp_check")
    g.edge("sp_check", "gpkg_files_06")
    g.edge("gpkg_files_06", "classe2", "Oui")
    g.edge("gpkg_files_06", "classe3", "Non")
    g.edge("classe1", "gpkg_files_07")
    g.edge("classe2", "gpkg_files_07")
    g.edge("classe3", "gpkg_files_07")

    return g

# Génération et sauvegarde
g = create_optimized_data_flow_diagram()
save_high_quality_graph(
    g, f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_02_classification", scale_factor=4
)
g

In [ ]:
def create_optimized_data_flow_diagram():
    """Crée le diagramme détaillé de la classification finale."""
    g = Digraph("G", format="png")
    g.attr(
        rankdir="TB",
        bgcolor="transparent",
        fontname="Arial",
        pad="1.0",
        nodesep="1.0",
        ranksep="0.8",
        ratio="compress",
    )

    # Configuration des styles
    g.attr(
        "node",
        shape="box",
        style="rounded,filled",
        fontname="Arial",
        fontsize="11",
        margin="0.2",
        width="2.0",
        height="0.5",
    )

    # Définition des styles
    file_style = {"fillcolor": "#E3F2FD", "color": "#1565C0", "height": "0.4"}
    output_file_style = {
        "fillcolor": "#E3F2FD",
        "color": "#1565C0",
        "height": "0.4",
        "penwidth": "2.5",
        "style": "rounded,filled,bold",
    }
    process_style = {"fillcolor": "#F1F8E9", "color": "#558B2F", "height": "0.4"}
    validation_style = {"fillcolor": "#FFF3E0", "color": "#EF6C00", "shape": "diamond"}
    action_style = {"fillcolor": "#E8EAF6", "color": "#3949AB", "height": "0.4"}
    action_style_2 = {"fillcolor": "#FFEBEE", "color": "#D32F2F", "height": "0.4"}

    # Création du cluster
    with g.subgraph(name="cluster_1") as c:
        c.attr(
            label="2. Classification des Toitures",
            style="rounded",
            color="#2A4D7E",
            margin="20",
        )

        # Organisation hiérarchique des nœuds
        with c.subgraph() as s:
            s.attr(rank="source")
            s.node("classify_start", "Classification\nToitures", **process_style)
            s.node("surface_check", "Surface > 2m²?", **validation_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node(
                "trou_check",
                "Polygone toiture avec\ntrou contenu dedans?",
                **validation_style
            )

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node("sp_check_avec_trou", "Superstructure?", **validation_style)
            s.node("sp_check_sans_trou", "Superstructure?", **validation_style)

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node(
                "classe2a",
                "Classe 2a\nPartiellement occupée\nPas de trou\nSuperstructure",
                **action_style
            )
            s.node(
                "classe2b",
                "Classe 2b:\nPartiellement occupée\nTrou\nSuperstructure",
                **action_style
            )
            s.node(
                "classe3a",
                "Classe 3a:\nLibre\nPas de trou\nPas de superstructure",
                **action_style
            )
            s.node(
                "classe3b",
                "Classe 3b:\nLibre\nTrou\nPas de superstructure",
                **action_style
            )

        with c.subgraph() as s:
            s.attr(rank="same")
            s.node(
                "classe2a_sans_sp",
                "Action sur image:\nSupprimer SP de l'image",
                **action_style_2
            )
            s.node("classe2a_avec_sp", "Pas d'action sur image", **action_style_2)
            s.node(
                "classe3b_sans_trou",
                "Action sur image:\nSupprimer trou de l'image",
                **action_style_2
            )
            s.node("classe3b_avec_trou", "Pas d'action sur image", **action_style_2)

        with c.subgraph() as s:
            s.attr(rank="sink")
            s.node("classe1", "Classe 1:\nTotalement occupée", **output_file_style)
            s.node("classe2", "Classe 2:\nPartiellement occupée", **output_file_style)
            s.node("classe3", "Classe 3:\nLibre", **output_file_style)

    # Ajout des connexions
    g.edge(
        "classify_start",
        "surface_check",
        constraint="true",
        tailport="e",
        headport="w",
    )
    g.edge("surface_check", "classe1", " ≤ 2m²")
    g.edge("surface_check", "trou_check", " > 2m²")
    g.edge("trou_check", "sp_check_sans_trou", " Non")
    g.edge("trou_check", "sp_check_avec_trou", " Oui")
    g.edge("sp_check_avec_trou", "classe2b", " Oui")
    g.edge("sp_check_avec_trou", "classe3b", " Non")
    g.edge("sp_check_sans_trou", "classe2a", " Oui")
    g.edge("sp_check_sans_trou", "classe3a", " Non")
    g.edge("classe2a", "classe2a_sans_sp")
    g.edge("classe2a", "classe2a_avec_sp")
    g.edge("classe3b", "classe3b_sans_trou")
    g.edge("classe3b", "classe3b_avec_trou")
    g.edge("classe2a_sans_sp", "classe3")
    g.edge("classe2a_avec_sp", "classe2")
    g.edge("classe3b_sans_trou", "classe3")
    g.edge("classe3b_avec_trou", "classe2")
    g.edge("classe3a", "classe3")
    g.edge("classe2b", "classe2")

    return g

# Génération et sauvegarde
g = create_optimized_data_flow_diagram()
save_high_quality_graph(
    g, f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_02a_classification_finale", scale_factor=4
)
g

## Préparer données
### Chargement des données

In [ ]:
def load_and_prepare_geodata(filepath, layer_name=None):
    """
    Charge et prépare les données géospatiales avec validation.
    
    Parameters:
        filepath (str): Chemin vers le fichier de données
        layer_name (str, optional): Nom de la couche si source multi-couches
        
    Returns:
        GeoDataFrame: Données nettoyées et validées
    """
    print(f"\nChargement: {filepath}")
    
    try:
        # Lecture du fichier
        if layer_name:
            gdf = gpd.read_file(filepath, layer=layer_name)
        else:
            gdf = gpd.read_file(filepath)
            
        initial_count = len(gdf)
        print(f"Nombre initial d'entités: {initial_count}")
        
        # Nettoyage et validation
        # Validation des géométries
        gdf['geometry'] = gdf['geometry'].make_valid()
        
        # Suppression des géométries vides
        gdf = gdf[~gdf['geometry'].is_empty]
        
        # Réindexation
        gdf = gdf.reset_index(drop=True)
        
        # Vérification du CRS
        if gdf.crs is None:
            print("Attention: Aucun CRS trouvé dans les données")
        else:
            print(f"CRS: {gdf.crs}")
        
        # Informations sur les géométries
        print("Types de géométries:", gdf.geometry.geom_type.unique())
        print(f"Nombre d'entités valides: {len(gdf)}")
        
        if len(gdf) != initial_count:
            print(f"{initial_count - len(gdf)} entités invalides/vides supprimées")
        
        # Vérification des types de géométries mixtes
        geom_types = gdf.geometry.geom_type.unique()
        if len(geom_types) > 1:
            print(f"Attention: Types de géométries mixtes détectés: {geom_types}")
        
        # Optimisation mémoire - conversion en catégories
        for col in gdf.select_dtypes(include=['object']).columns:
            if col != 'geometry' and gdf[col].nunique() < len(gdf) * 0.5:
                gdf[col] = gdf[col].astype('category')
        
        return gdf
    
    except Exception as e:
        print(f"Erreur lors du chargement de {filepath}: {str(e)}")
        raise

# Suppression des avertissements non critiques
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)

print("Chargement des datasets...")

gdf_toiture = load_and_prepare_geodata(CAD_BATIMENT_HORSOL_TOIT_GPKG_PATH)
gdf_toiture_sp = load_and_prepare_geodata(CAD_BATIMENT_HORSOL_TOIT_SP_GPKG_PATH)
gdf_cad_batiment_horsol = load_and_prepare_geodata(CAD_BATIMENT_HORSOL_GPKG_PATH)
gdf_cad_commune = load_and_prepare_geodata(CAD_COMMUNE_GPKG_PATH)

# Vérification de la cohérence des CRS
crs_list = [
    gdf_toiture.crs,
    gdf_toiture_sp.crs,
    gdf_cad_batiment_horsol.crs,
    gdf_cad_commune.crs
]

if len(set(crs_list)) > 1:
    print("\nAttention: CRS différents détectés entre les datasets:")
    print("gdf_toiture CRS:", gdf_toiture.crs)
    print("gdf_toiture_sp CRS:", gdf_toiture_sp.crs)
    print("gdf_cad_batiment_horsol CRS:", gdf_cad_batiment_horsol.crs)
    print("gdf_cad_commune CRS:", gdf_cad_commune.crs)

# Résumé du chargement
print("\nRésumé du chargement des données:")
print(f"gdf_toiture: {len(gdf_toiture)} entités")
print(f"gdf_toiture_sp: {len(gdf_toiture_sp)} entités")
print(f"gdf_cad_batiment_horsol: {len(gdf_cad_batiment_horsol)} entités")
print(f"gdf_cad_commune: {len(gdf_cad_commune)} entités")



### Correction des types de données

In [ ]:
# Conversion des dates depuis timestamp millisecondes
gdf_toiture["date_leve"] = pd.to_datetime(gdf_toiture["date_leve"], unit="ms")
gdf_toiture_sp["date_leve"] = pd.to_datetime(gdf_toiture_sp["date_leve"], unit="ms")

# Nettoyage des identifiants globaux (suppression des accolades)
gdf_toiture["globalid"] = (
    gdf_toiture["globalid"].str.replace("{", "").str.replace("}", "")
)
gdf_toiture_sp["globalid"] = (
    gdf_toiture_sp["globalid"].str.replace("{", "").str.replace("}", "")
)
gdf_cad_batiment_horsol["globalid"] = (
    gdf_cad_batiment_horsol["globalid"].str.replace("{", "").str.replace("}", "")
)
gdf_cad_commune["globalid"] = (
    gdf_cad_commune["globalid"].str.replace("{", "").str.replace("}", "")
)

# Affichage des échantillons et types de données
display(gdf_toiture.head(2))
display(gdf_toiture.dtypes)
display(gdf_toiture_sp.head(2))
display(gdf_toiture_sp.dtypes)
display(gdf_cad_batiment_horsol.head(2))
display(gdf_cad_batiment_horsol.dtypes)
display(gdf_cad_commune.head(2))
display(gdf_cad_commune.dtypes)

### Validation des géométries

In [ ]:
def verify_commune_data(gdf):
    """Vérifie l'intégrité des données communales avec focus sur Céligny."""
    print("=== Vérification des données communales ===")
    print(f"\nNombre total de communes: {len(gdf)}")
    print("\nTypes de géométries présents:")
    print(gdf.geometry.type.value_counts())

    print("\nVérification de Céligny:")
    celigny = gdf[gdf["commune"] == "Céligny"]
    print(f"Nombre d'enregistrements pour Céligny: {len(celigny)}")
    if len(celigny) > 0:
        print("Type de géométrie pour Céligny:", celigny.geometry.type.values)
        print("Bounds de Céligny:", celigny.total_bounds)

    # Visualisation
    fig, ax = plt.subplots(figsize=(15, 10))
    gdf.plot(ax=ax, alpha=0.5)
    if len(celigny) > 0:
        celigny.plot(ax=ax, color="red", alpha=0.7)
    plt.title("Vérification des communes (Céligny en rouge)")
    plt.show()

    return celigny

celigny = verify_commune_data(gdf_cad_commune)

In [ ]:
# Vérification des CRS
print("CRS des couches:")
print(f"gdf_toiture CRS: {gdf_toiture.crs}")
print(f"gdf_toiture_sp CRS: {gdf_toiture_sp.crs}")
print(f"gdf_cad_commune CRS: {gdf_cad_commune.crs}")
print(f"gdf_cad_batiment_horsol CRS: {gdf_cad_batiment_horsol.crs}")

# Vérification de la validité des géométries
print("\nValidité des géométries:")
print(f"Géométries invalides dans gdf_toiture: {sum(~gdf_toiture.is_valid)}")
print(f"Géométries invalides dans gdf_toiture_sp: {sum(~gdf_toiture_sp.is_valid)}")
print(f"Géométries invalides dans gdf_cad_commune: {sum(~gdf_cad_commune.is_valid)}")
print(
    f"Géométries invalides dans gdf_cad_batiment_horsol: {sum(~gdf_cad_batiment_horsol.is_valid)}"
)

print("\nGéométries invalides après correction:")
print(f"Géométries invalides dans gdf_toiture: {sum(~gdf_toiture.is_valid)}")
print(f"Géométries invalides dans gdf_toiture_sp: {sum(~gdf_toiture_sp.is_valid)}")
print(f"Géométries invalides dans gdf_cad_commune: {sum(~gdf_cad_commune.is_valid)}")
print(
    f"Géométries invalides dans gdf_cad_batiment_horsol: {sum(~gdf_cad_batiment_horsol.is_valid)}"
)

### Filtrage des polygones hors canton

In [ ]:
def fast_spatial_filter(gdf_toiture, gdf_cad_commune):
    """
    Filtrage spatial optimisé des polygones de toiture utilisant prepared geometry
    et index spatial.

    Parameters:
        gdf_toiture (GeoDataFrame): Polygones de toiture
        gdf_cad_commune (GeoDataFrame): Limites communales cadastrales

    Returns:
        tuple: (gdf_filtré, gdf_exclu)
    """
    # Création d'un polygone unique depuis les limites communales
    if len(gdf_cad_commune) > 1:
        boundary = gdf_cad_commune.geometry.union_all()
    else:
        boundary = gdf_cad_commune.geometry.iloc[0]

    # Prepared geometry pour des opérations plus rapides
    prepared_boundary = prep(boundary)

    # Utilisation de l'index spatial
    spatial_index = gdf_toiture.sindex

    # Candidats potentiels via l'index spatial
    possible_matches_index = list(spatial_index.intersection(boundary.bounds))
    possible_matches = gdf_toiture.iloc[possible_matches_index]

    # Filtrage final avec prepared geometry
    mask = possible_matches.geometry.apply(lambda x: prepared_boundary.intersects(x))
    filtered = possible_matches[mask]

    # Géométries exclues
    excluded = gdf_toiture[~gdf_toiture.index.isin(filtered.index)]

    return filtered, excluded

# Filtrage pour gdf_toiture
gdf_toiture_1_filtre_canton, gdf_toiture_1_filtre_canton_elimines = fast_spatial_filter(gdf_toiture, gdf_cad_commune)
print(
    f"Nombre de polygones hors canton dans gdf_toiture : {len(gdf_toiture_1_filtre_canton_elimines)}"
)

# Filtrage pour gdf_toiture_sp
gdf_toiture_sp_1_filtre_canton, gdf_toiture_sp_1_filtre_canton_elimines = fast_spatial_filter(gdf_toiture_sp, gdf_cad_commune)
print(
    f"Nombre de polygones hors canton dans gdf_toiture_sp : {len(gdf_toiture_sp_1_filtre_canton_elimines)}"
)

In [ ]:
def visualize_problem_area1(
    gdf_toiture,
    gdf_cad_commune,
    gdf_toiture_1_filtre_canton,
    eliminated_polys,
    gdf_name,
):
    """Visualise les zones filtrées avec focus sur les polygones éliminés."""
    sns.set_style("whitegrid")
    fig, ax = plt.subplots(figsize=(6.5, 6.5), dpi=300)

    plt.title(
        "Visualisation des zones filtrées hors canton",
        pad=10,
        fontsize=10,
        fontweight="bold",
    )

    # Création des éléments de légende
    legend_elements = []

    # Tracé des couches
    # Communes
    commune_plot = gdf_cad_commune.plot(
        ax=ax,
        color="grey",
        alpha=0.3,
        label="Limites communales",
        zorder=1,
    )
    legend_elements.append(
        Patch(facecolor="grey", alpha=0.3, label="Limites communes GE")
    )

    # Toitures retenues
    retained_plot = gdf_toiture_1_filtre_canton.plot(
        ax=ax,
        color="blue",
        edgecolor="darkblue",
        linewidth=0.3,
        alpha=0.9,
        label="Toitures retenues pour l'analyse",
        zorder=4,
    )
    legend_elements.append(
        Patch(facecolor="blue", alpha=1, label="Toitures retenues pour l'analyse")
    )

    # Toitures éliminées
    eliminated_plot = eliminated_polys.plot(
        ax=ax,
        color="red",
        alpha=1,
        label="Toitures exclues de l'analyse",
        zorder=3,
    )
    legend_elements.append(
        Patch(facecolor="red", alpha=1, label="Toitures exclues de l'analyse")
    )

    # Limites des communes
    boundary_plot = gdf_cad_commune.boundary.plot(
        ax=ax,
        color="#2C3E50",
        linewidth=0.8,
        linestyle="--",
        zorder=4,
    )
    legend_elements.append(
        Line2D([0], [0], color="#2C3E50", linestyle="--", label="Limites des communes")
    )

    # Légende
    ax.legend(
        handles=legend_elements,
        title="Légende",
        title_fontsize=8,
        fontsize=8,
        loc="upper left",
        facecolor="white",
        edgecolor="#CCCCCC",
    )

    # Configuration des axes
    ax.grid(True, linestyle=":", alpha=0.3)
    ax.set_xlabel("Coordonnées X", fontsize=8)
    ax.set_ylabel("Coordonnées Y", fontsize=8)
    plt.tick_params(axis='both', labelsize=8)
    ax.xaxis.get_offset_text().set_fontsize(8)
    ax.yaxis.get_offset_text().set_fontsize(8)

    # Zoom sur les toitures éliminées
    if not eliminated_polys.empty:
        bounds = eliminated_polys.total_bounds
        margin = 100
        x_min, y_min, x_max, y_max = bounds
        width = x_max - x_min
        height = y_max - y_min
        margin_x = width * 0.1
        margin_y = height * 0.1
        ax.set_xlim(x_min - margin_x, x_max + margin_x)
        ax.set_ylim(y_min - margin_y, y_max + margin_y)
    else:
        plt.text(
            0.5,
            0.5,
            "Aucune toiture éliminée",
            horizontalalignment="center",
            verticalalignment="center",
            transform=ax.transAxes,
            fontsize=8,
        )

    plt.tight_layout()

    # Statistiques
    stats_text = (
        f"Nombre total de toitures: {len(gdf_toiture)}\n"
        f"Toitures conservées: {len(gdf_toiture_1_filtre_canton)}\n"
        f"Toitures éliminées: {len(eliminated_polys)}"
    )
    plt.figtext(
        0.152,
        0.752,
        stats_text,
        fontsize=8,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="#CCCCCC"),
        horizontalalignment="left",
        verticalalignment="center",
    )

    # Sauvegarde
    output_file = f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_01_visu_zone_hors_canton.png"
    plt.savefig(output_file, bbox_inches="tight", dpi=300)

    plt.show()

plt.close()
# Visualisation
visualize_problem_area1(
    gdf_toiture,
    gdf_cad_commune,
    gdf_toiture_1_filtre_canton,
    gdf_toiture_1_filtre_canton_elimines,
    "CAD_BATIMENT_HORSOL_TOIT",
)

In [ ]:
def visualize_problem_area2(
    gdf_toiture,
    gdf_cad_commune,
    gdf_toiture_1_filtre_canton,
    eliminated_polys,
    gdf_name,
):
    """Visualise les zones filtrées avec vue détaillée."""
    sns.set_style("whitegrid")
    fig, ax = plt.subplots(figsize=(10,10))

    plt.title(
        f"Visualisation des zones filtrées pour {gdf_name}",
        pad=20,
        fontsize=14,
        fontweight="bold",
    )

    # Fond de carte (communes)
    gdf_cad_commune.plot(
        ax=ax,
        color="#EEEEEE",
        alpha=0.3,
    )

    # Toitures retenues (couche inférieure)
    if not gdf_toiture_1_filtre_canton.empty:
        gdf_toiture_1_filtre_canton.plot(
            ax=ax,
            color="#2E86C1",
            alpha=0.4,
            linewidth=0.5,
            edgecolor="#1B4F72",
        )

    # Toitures éliminées (couche supérieure)
    eliminated_polys.plot(
        ax=ax,
        color="#E74C3C",
        alpha=0.8,
        linewidth=1.5,
        edgecolor="#943126",
    )

    # Limites des communes
    gdf_cad_commune.boundary.plot(
        ax=ax,
        color="#2C3E50",
        linewidth=0.8,
        linestyle="--",
    )

    # Légende
    legend_elements = [
        Patch(
            facecolor="#2E86C1",
            alpha=0.4,
            edgecolor="#1B4F72",
            linewidth=0.5,
            label="Toitures retenues pour l'analyse",
        ),
        Patch(
            facecolor="#E74C3C",
            alpha=0.8,
            edgecolor="#943126",
            linewidth=1,
            label="Toitures exclues de l'analyse",
        ),
        Line2D([0], [0], color="#2C3E50", linestyle="--", label="Limites des communes"),
    ]

    ax.legend(
        handles=legend_elements,
        title="Légende",
        title_fontsize=12,
        fontsize=10,
        loc="upper left",
        facecolor="white",
        edgecolor="#CCCCCC",
    )

    # Configuration des axes
    ax.grid(True, linestyle=":", alpha=0.3)
    ax.set_xlabel("Coordonnées X", fontsize=10)
    ax.set_ylabel("Coordonnées Y", fontsize=10)

    # Zone de visualisation basée sur les toitures éliminées
    if not eliminated_polys.empty:
        bounds = eliminated_polys.total_bounds
        x_min, y_min, x_max, y_max = bounds
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min
        margin_factor = 0.5
        zoom_width = width * (1 + margin_factor)
        zoom_height = height * (1 + margin_factor)
        ax.set_xlim(center_x - zoom_width / 2, center_x + zoom_width / 2)
        ax.set_ylim(center_y - zoom_height / 2, center_y + zoom_height / 2)

    plt.tight_layout()

    # Statistiques
    stats_text = (
        f"Nombre total de toitures: {len(gdf_toiture)}\n"
        f"Toitures conservées: {len(gdf_toiture_1_filtre_canton)}\n"
        f"Toitures éliminées: {len(eliminated_polys)}"
    )
    plt.figtext(
        0.12,
        0.815,
        stats_text,
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="#CCCCCC"),
        horizontalalignment="left",
        verticalalignment="center",
    )

    plt.show()

try:
    visualize_problem_area2(
        gdf_toiture_sp,
        gdf_cad_commune,
        gdf_toiture_sp_1_filtre_canton,
        gdf_toiture_sp_1_filtre_canton_elimines,
        "CAD_BATIMENT_HORSOL_TOIT_SP",
    )
except Exception as e:
    print(f"Erreur lors de la visualisation : {e}")

In [ ]:
# Sauvegarde des résultats en Parquet
gdf_toiture_1_filtre_canton.to_parquet(
    f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_1_filtre_canton.parquet"
)
gdf_toiture_sp_1_filtre_canton.to_parquet(
    f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_sp_1_filtre_canton.parquet"
)

### Filtrage par EGID valide

In [ ]:
# Identification des EGID invalides (inférieurs au minimum des bâtiments)
egid_bizarre = gdf_cad_batiment_horsol["egid"].min()
print("EGID minimum dans gdf_cad_batiment_horsol:", egid_bizarre)
print("Nombre de toitures avec un EGID invalide:")
print(f"\t{gdf_toiture_1_filtre_canton[gdf_toiture_1_filtre_canton['egid'] < egid_bizarre].shape[0]} \
dans gdf_toiture_1_filtre_canton")
print(f"\t{gdf_toiture_sp_1_filtre_canton[gdf_toiture_sp_1_filtre_canton['egid'] < egid_bizarre].shape[0]} \
dans gdf_toiture_sp_1_filtre_canton")
print(f"\t{gdf_cad_batiment_horsol[gdf_cad_batiment_horsol['egid'] < egid_bizarre].shape[0]} \
dans gdf_cad_batiment_horsol")

filtre_egid_toiture = gdf_toiture_1_filtre_canton[
    gdf_toiture_1_filtre_canton["egid"] < egid_bizarre
]

In [ ]:
# Application du filtre EGID
gdf_toiture_2_filtre_egid = gdf_toiture_1_filtre_canton[
    gdf_toiture_1_filtre_canton["egid"] > gdf_cad_batiment_horsol["egid"].min()
]

gdf_toiture_sp_2_filtre_egid = gdf_toiture_sp_1_filtre_canton[
    gdf_toiture_sp_1_filtre_canton["egid"] > gdf_cad_batiment_horsol["egid"].min()
]

# Sauvegarde des résultats
gdf_toiture_2_filtre_egid.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_2_filtre_egid.parquet"
)
gdf_toiture_sp_2_filtre_egid.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_sp_2_filtre_egid.parquet"
)

### Catégories SIA

In [ ]:
def assign_sia_category(destination):
    """
    Attribue une catégorie SIA 380/1:2009 selon la destination du bâtiment.
    
    Parameters:
        destination (str): Destination du bâtiment
        
    Returns:
        str: Catégorie SIA (I à XII) ou 'NA'
    """
    dest = str(destination).lower()

    # Catégorie I - Habitat collectif
    habitat_collectif = [
        "hab plusieurs logements",
        "résidence meublée",
        "foyer",
        "hôtel",
        "autre héberg. collectif",
        "etab. pénitenciaire",
        "caserne",
        "internat",
        "hab. - rez activités",
        "habitation - activités",
    ]
    if any(term in dest for term in habitat_collectif):
        return "I habitat collectif"

    # Catégorie II - Habitat individuel
    habitat_individuel = [
        "habitation un logement",
        "hab. deux logements",
        ]
    if any(term in dest for term in habitat_individuel):
        return "II habitat individuel"

    # Catégorie III - Administration
    administration = [
        "bureaux",
        "bureaux des oig",
        "administration",
        "administrations publiques",
        "mairie",
        "poste",
        "mission permanente",
        "consulat",
        "douane",
        "police",
        "bibliothèque",
        "musée",
        "onu",
        "central de télécom.",
        "cabine t+t",
        "installation de téléphonie mobile",
        "autre équipement collectif",
    ]
    if any(term in dest for term in administration):
        return "III administration"

    # Catégorie IV - Écoles
    ecoles = [
        "école primaire",
        "ecole privée",
        "collège",
        "université",
        "jardin d'enfants",
        "conservatoire musique",
        "autre école",
        "ecole primaire",
    ]
    if any(term in dest for term in ecoles):
        return "IV écoles"

    # Catégorie V - Commerce
    commerce = [
        "commerce",
        "centre commercial",
        "halle d'exposition",
        "port-franc"]
    if any(term in dest for term in commerce):
        return "V commerce"

    # Catégorie VI - Restauration
    restauration = ["restaurant", "divertissement"]
    if any(term in dest for term in restauration):
        return "VI restauration"

    # Catégorie VII - Lieux de rassemblement
    rassemblement = [
        "temple",
        "chapelle",
        "église",
        "synagogue",
        "mosquée",
        "autre lieu de culte",
        "cinéma",
        "théâtre",
        "salle de spectacle",
        "salle communale",
        "centre de loisirs",
        "autre bât. de loisirs",
        "wc public",
        "eglise",
        "parking public",
    ]
    if any(term in dest for term in rassemblement):
        return "VII lieux de rassemblement"

    # Catégorie VIII - Hôpitaux
    hopitaux = [
        "hôpital,clinique",
        "etablissement de soins",
        "hôpital, clinique",
        "ems",
    ]
    if any(term in dest for term in hopitaux):
        return "VIII hôpitaux"

    # Catégorie IX - Industrie
    industrie = [
        "usine",
        "atelier",
        "garage privé",
        "garage",
        "cern",
        "ferme",
        "voirie-entretien",
        "gare",
        "service du feu",
        "arsenal",
        "station-service",
        "station d'épuration",
        "bât. électricité sig",
        "bâtiment électricité",
        "instal. tech. élec. sig",
        "instal. tech. élec.",
        "bâtiment gaz sig",
        "bâtiment gaz",
        "instal. tech. gaz sig",
        "instal. tech.gaz",
        "bâtiment eau sig",
        "bâtiment eau",
        "instal. tech. eau sig",
        "instal. tech. eau",
        "ouvrage aéroportuaire",
        "sécurité civile",
        "dépôt tpg",
        "chantier naval",
        "autre bâtiment de prod. agricole",
        "installation de chauffage",
        "cheminée",
        "poulailler",
        "serre",
        "porcherie",
        "citerne gaz",
        "compostage",
        "chauffage à distance",
        "citerne mazout",
        "installation de climatisation",
        "réservoir",
        "ecurie",
    ]
    if any(term in dest for term in industrie):
        return "IX industrie"

    # Catégorie X - Dépôts
    depots = [
        "dépôt",
        "hangar",
        "hangar agricole",
        "silo",
        "autre bât. de prod. agricole",
        "déchetterie",
        "autre bât. < 20 m2",
        "autre bât. 20m2 et plus",
        "autre bât. d'activités",
    ]
    if any(term in dest for term in depots):
        return "X dépôts"

    # Catégorie XI - Installations sportives
    sport = [
        "salle de sport",
        "centre sportif",
        "stade",
        "stand de tir",
        "manège",
        "patinoire",
    ]
    if any(term in dest for term in sport):
        return "XI installations sportives"

    # Catégorie XII - Piscines couvertes
    piscines = [
        "piscine",
    ]
    if any(term in dest for term in piscines):
        return "XII piscines couvertes"

    return "NA"

# Application de la catégorisation
gdf_cad_batiment_horsol['sia_cat'] = gdf_cad_batiment_horsol['destination'].apply(assign_sia_category)

display(gdf_cad_batiment_horsol[gdf_cad_batiment_horsol["sia_cat"] == "NA"]["destination"].str.lower().unique())

In [ ]:
print(gdf_cad_batiment_horsol["sia_cat"].value_counts())
print(f"Nombre de valeurs manquantes dans sia_cat: {gdf_cad_batiment_horsol["sia_cat"].isna().sum()}")
assert gdf_cad_batiment_horsol["sia_cat"].isna().sum() == 0

In [ ]:
def format_number(num):
    """Formate un nombre avec apostrophe comme séparateur de milliers."""
    return f"{num:,}".replace(",", "'")

def create_sia_categories_analysis(df, output_path=None):
    """
    Crée une analyse complète des catégories SIA avec visualisation et statistiques.

    Parameters:
        df (DataFrame): DataFrame contenant les catégories SIA
        output_path (str, optional): Chemin de sauvegarde

    Returns:
        tuple: (figure, dataframe_statistiques)
    """
    sns.set_style("whitegrid")
    
    # Définition des noms de catégories et palette de couleurs
    sia_categories = {
        "I habitat collectif": "Habitat collectif",
        "II habitat individuel": "Habitat individuel", 
        "III administration": "Administration",
        "IV écoles": "Écoles",
        "V commerce": "Commerce",
        "VI restauration": "Restauration",
        "VII lieux de rassemblement": "Lieux de rassemblement",
        "VIII hôpitaux": "Hôpitaux",
        "IX industrie": "Industrie",
        "X dépôts": "Dépôts",
        "XI installations sportives": "Installations sportives",
        "XII piscines couvertes": "Piscines couvertes",
    }

    # Palette de couleurs pastel
    pastel_colors = sns.color_palette("pastel", len(sia_categories))
    sia_colors = dict(zip(sorted(sia_categories.keys()), pastel_colors))

    # Calcul des statistiques
    total = len(df)
    value_counts = df["sia_cat"].value_counts()
    percentages = (value_counts / total * 100).round(1)

    # Création de la figure
    fig, ax = plt.subplots(figsize=(6.5, 5.5))
    
    # Graphique en barres
    ordered_categories = sorted(df["sia_cat"].unique())
    bars = ax.bar(range(len(ordered_categories)), 
                  [value_counts[cat] for cat in ordered_categories],
                  color=[sia_colors[cat] for cat in ordered_categories],
                  edgecolor='white', linewidth=0.5)

    # Configuration du titre et des labels
    ax.set_title(f"Distribution des catégories SIA\n({format_number(total)} bâtiments)", 
                fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel("Catégorie SIA", fontsize=10)
    ax.set_ylabel("Nombre de bâtiments (EGID)", fontsize=10)

    # Configuration de la grille
    ax.grid(False)
    ax.set_axisbelow(True)
    
    # Configuration de l'axe X
    ax.set_xticks(range(len(ordered_categories)))
    ax.set_xticklabels(ordered_categories, rotation=45, ha='right', fontsize=10)

    # Ajout des valeurs sur les barres
    for i, (bar, cat) in enumerate(zip(bars, ordered_categories)):
        height = bar.get_height()
        percentage = percentages[cat]
        
        if percentage >= 1.0:
            label = f"{format_number(int(height))}\n({percentage:.1f}%)"
        else:
            label = f"{format_number(int(height))}"
            
        ax.text(bar.get_x() + bar.get_width()/2, height + total*0.005,
                label, ha='center', va='bottom', 
                fontsize=9)

    ax.set_ylim(0, 35000)

    # Suppression des bordures
    sns.despine(ax=ax, top=True, right=True)
    ax.yaxis.grid(False)

    plt.tight_layout()
    
    # Sauvegarde si chemin fourni
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches="tight", 
                   facecolor="white", edgecolor='none')

    # Création du tableau récapitulatif
    summary_df = pd.DataFrame({
        "Catégorie": value_counts.index,
        "Description": value_counts.index.map(sia_categories),
        "Nombre": [format_number(int(x)) for x in value_counts.values],
        "Pourcentage": [f"{p:.1f}%" for p in percentages.round(1)],
    })

    return fig, summary_df

fig, summary = create_sia_categories_analysis(
    gdf_cad_batiment_horsol, f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_04_cad_batiment_horsol_sia_distribution.png"
)
print("\nTableau récapitulatif:")
print(summary.to_string(index=False))

In [ ]:
# Fusion et ajout de sia_cat aux toitures
gdf_toiture_3_ajout_cat_sia = gdf_toiture_2_filtre_egid.merge(
    gdf_cad_batiment_horsol[["egid", "sia_cat"]], how="left", on="egid"
)
display(gdf_toiture_3_ajout_cat_sia.head(2))
display(gdf_toiture_3_ajout_cat_sia[gdf_toiture_3_ajout_cat_sia["sia_cat"] == "NA"]["egid"].unique())

# Sauvegarde
gdf_toiture_3_ajout_cat_sia.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_3_ajout_cat_sia.parquet"
)

In [ ]:
display(gdf_toiture_3_ajout_cat_sia)

In [ ]:
assert gdf_toiture_3_ajout_cat_sia["sia_cat"].isna().sum() == 0

## Classification automatique des toitures (option explorée mais pas retenue pour la méthodologie)

### Schéma de classification

In [ ]:
def create_optimized_data_flow_diagram():
    """Crée le diagramme détaillé du processus de classification."""
    g = Digraph("G", format="png")
    g.attr(
        rankdir="TB",
        bgcolor="transparent",
        fontname="Arial",
        pad="1.0",
        nodesep="1.0",
        ranksep="0.8",
        ratio="compress",
    )

    # Configuration des styles
    g.attr(
        "node",
        shape="box",
        style="rounded,filled",
        fontname="Arial",
        fontsize="11",
        margin="0.2",
        width="2.0",
        height="0.5",
    )

    # Configuration des arêtes
    g.attr(
        "edge", fontname="Arial", fontsize="10", color="#2A4D7E", fontcolor="#505050"
    )

    # Styles de nœuds
    start_style = {
        "fillcolor": "#F8F9FA",
        "color": "#2A4D7E",
        "style": "rounded,filled",
    }
    decision_style = {
        "shape": "diamond",
        "fillcolor": "#E5ECF6",
        "color": "#2A4D7E",
        "margin": "0.2",
        "style": "filled",
    }
    class1_style = {
        "fillcolor": "#E1F0E5",
        "color": "#2E7D32",
        "style": "rounded,filled",
    }
    class2_style = {
        "fillcolor": "#FFF3E0",
        "color": "#E65100",
        "style": "rounded,filled",
    }
    class3_style = {
        "fillcolor": "#F8E6FF",
        "color": "#6A1B9A",
        "style": "rounded,filled",
    }

    # Création du cluster
    with g.subgraph(name="cluster_0") as c:
        c.attr(
            label="Classification des toitures",
            style="rounded",
            color="#2A4D7E",
            margin="20",
            fontsize="16",
        )

        # Création des nœuds
        c.node("toiture", "Polygones\ngdf_toiture_3_ajout_cat_sia.gpkg", **start_style)
        c.node("surface", "Polygone > 2m²?", **decision_style)
        c.node("classe1", "Classe 1:\ntoiture totalement occupée", **class1_style)
        c.node("trou", "Polygone avec\nTrou totalement\ninclus?", **decision_style)
        c.node("sp_libre", "Elements SP\nà l'intérieur?", **decision_style)
        c.node("sp_partiel", "Elements SP\nà l'intérieur?", **decision_style)
        c.node(
            "classe2a",
            "Classe 2a:\nToiture partiellement occupée\nSans trou",
            **class2_style
        )
        c.node(
            "classe2b",
            "Classe 2b:\nToiture partiellement occupée\nAvec trou",
            **class2_style
        )
        c.node("classe3a", "Classe 3a:\nToiture libre\nSans trou", **class3_style)
        c.node("classe3b", "Classe 3b:\nToiture libre\nAvec trou", **class3_style)

        # Ajout des arêtes
        c.edge("toiture", "surface")
        c.edge("surface", "classe1", " non")
        c.edge("surface", "trou", " oui")
        c.edge("trou", "sp_partiel", " oui")
        c.edge("trou", "sp_libre", " non")
        c.edge("sp_libre", "classe2a", " oui")
        c.edge("sp_libre", "classe3a", " non")
        c.edge("sp_partiel", "classe2b", " oui")
        c.edge("sp_partiel", "classe3b", " non")

    return g

# Génération et sauvegarde
g = create_optimized_data_flow_diagram()
save_high_quality_graph(
    g, f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_03_classification_detail_classes", scale_factor=4
)
g

### Identification des toitures de moins de 2 m²

In [ ]:
gdf_toiture_4_polgone_2m2 = gdf_toiture_3_ajout_cat_sia.copy()

# Création de la colonne booléenne pour surface > 2m²
LIMITE_SURFACE = 2
gdf_toiture_4_polgone_2m2["shape_area_sup_2m2"] = (
    gdf_toiture_4_polgone_2m2["SHAPE__Area"] > LIMITE_SURFACE
)

display(gdf_toiture_4_polgone_2m2.head(2))

# Sauvegarde du résultat intermédiaire
gdf_toiture_4_polgone_2m2.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_4_polgone_2m2.parquet"
)

### Détection des trous dans les toitures

In [ ]:
def is_fully_contained_hole(exterior, interior, min_area=2):
    """
    Vérifie si un trou est entièrement contenu dans le polygone extérieur.

    Parameters:
        exterior (LinearRing): Limite extérieure du polygone
        interior (LinearRing): Anneau intérieur (trou)
        min_area (float): Surface minimale pour considérer un trou

    Returns:
        bool: True si le trou est valide et contenu
    """
    exterior_poly = Polygon(exterior)
    interior_poly = Polygon(interior)

    # Vérification du confinement et de la surface minimale
    if exterior_poly.contains(interior_poly) and interior_poly.area >= min_area:
        return True
    return False

def analyze_geometry_holes(geometry, min_area=2):
    """
    Analyse les trous dans une géométrie.

    Parameters:
        geometry: Objet géométrique Shapely (Polygon ou MultiPolygon)
        min_area (float): Surface minimale pour un trou valide

    Returns:
        dict: Résultats de l'analyse des trous
    """
    if geometry is None or not geometry.is_valid:
        return {
            "has_holes": False,
            "hole_count": 0,
            "hole_areas": [],
            "total_hole_area": 0,
            "valid_geometry": False,
        }

    contained_holes = []
    total_holes = 0

    if isinstance(geometry, Polygon):
        exterior = geometry.exterior
        for interior in geometry.interiors:
            total_holes += 1
            if is_fully_contained_hole(exterior, interior, min_area):
                contained_holes.append(Polygon(interior))

    elif isinstance(geometry, MultiPolygon):
        for poly in geometry.geoms:
            exterior = poly.exterior
            for interior in poly.interiors:
                total_holes += 1
                if is_fully_contained_hole(exterior, interior, min_area):
                    contained_holes.append(Polygon(interior))

    hole_areas = [hole.area for hole in contained_holes]

    return {
        "has_holes": len(contained_holes) > 0,
        "hole_count": len(contained_holes),
        "total_holes_checked": total_holes,
        "hole_areas": hole_areas,
        "total_hole_area": sum(hole_areas),
        "valid_geometry": True,
    }

def analyze_holes_in_gdf(gdf, geometry_column="geometry", min_area=200):
    """
    Analyse les trous dans toutes les géométries d'un GeoDataFrame.

    Parameters:
        gdf (GeoDataFrame): GeoDataFrame d'entrée
        geometry_column (str): Nom de la colonne géométrie
        min_area (float): Surface minimale pour un trou valide

    Returns:
        GeoDataFrame: GeoDataFrame avec colonnes d'analyse ajoutées
    """
    # Application de l'analyse à chaque géométrie
    results = gdf[geometry_column].apply(
        lambda geom: analyze_geometry_holes(geom, min_area=min_area)
    )

    # Extraction des résultats
    gdf["has_fully_contained_holes"] = results.apply(lambda x: x["has_holes"])
    gdf["fully_contained_hole_count"] = results.apply(lambda x: x["hole_count"])
    gdf["total_hole_area"] = results.apply(lambda x: x["total_hole_area"])
    gdf["valid_geometry"] = results.apply(lambda x: x["valid_geometry"])

    # Calcul du ratio surface trous/surface totale
    gdf["hole_area_ratio"] = gdf["total_hole_area"] / gdf[geometry_column].area

    return gdf

def find_significant_holes(gdf, min_hole_area_ratio=0.01):
    """
    Filtre les géométries avec des trous significatifs.

    Parameters:
        gdf (GeoDataFrame): GeoDataFrame avec analyse des trous
        min_hole_area_ratio (float): Ratio minimal trou/polygone

    Returns:
        GeoDataFrame: Géométries avec trous significatifs
    """
    return gdf[
        (gdf["has_fully_contained_holes"])
        & (gdf["valid_geometry"])
        & (gdf["hole_area_ratio"] >= min_hole_area_ratio)
    ]

# Application de l'analyse
gdf_toiture_5_hole = gdf_toiture_4_polgone_2m2.copy()
gdf_toiture_5_hole = analyze_holes_in_gdf(gdf_toiture_5_hole, min_area=2)

# Identification des trous significatifs
significant_holes = find_significant_holes(gdf_toiture_5_hole, min_hole_area_ratio=0.01)

# Affichage des résultats
print(f"Total géométries: {len(gdf_toiture_5_hole)}")
print(
    f"Géométries avec trous: {len(gdf_toiture_5_hole[gdf_toiture_5_hole['fully_contained_hole_count'] > 0])}"
)
print(f"Géométries avec trous significatifs: {len(significant_holes)}")

significant_holes.head(1).plot()

# Sauvegarde
significant_holes.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_5_hole.parquet"
)

### Analyse des superstructures dans les toitures

In [ ]:
gdf_toiture_6_contains_SP_svg_content = """<svg viewBox="0 0 800 580" xmlns="http://www.w3.org/2000/svg">
    <!-- Title -->
    <text x="400" y="30" text-anchor="middle" font-family="Arial" font-size="20" font-weight="bold">Spatial Relationships Between Roofs and SP Elements</text>
    <text x="400" y="55" text-anchor="middle" font-family="Arial" font-size="14">Analysis of Contains, Within, and Overlaps Cases</text>
    
    <!-- Row 1: Basic Cases -->
    <g transform="translate(40,80)">
        <g>
            <!-- Contains -->
            <rect x="0" y="0" width="150" height="100" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
            <rect x="30" y="20" width="90" height="60" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
            <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Contains</text>
            <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">sp_contains = True</text>
            <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">sp_relation_type = "contains"</text>
        </g>
    </g>
    
    <g transform="translate(240,80)">
        <!-- Within -->
        <rect x="0" y="0" width="150" height="100" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <rect x="30" y="20" width="90" height="60" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
        <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Within</text>
        <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">sp_within = True</text>
        <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">sp_relation_type = "within"</text>
    </g>

    <g transform="translate(440,80)">
        <!-- Overlaps -->
        <rect x="0" y="0" width="150" height="100" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
        <rect x="75" y="20" width="90" height="60" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Overlaps</text>
        <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">sp_overlaps = True</text>
        <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">sp_relation_type = "overlaps"</text>
    </g>

    <g transform="translate(640,80)">
        <!-- Multiple Elements -->
        <rect x="0" y="0" width="150" height="100" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
        <rect x="20" y="20" width="40" height="30" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <rect x="90" y="50" width="40" height="30" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Multiple Elements</text>
        <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">sp_element_count = 2</text>
        <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">has_sp_elements = True</text>
    </g>

    <!-- Row 2: Complex Cases -->
    <g transform="translate(40,280)">
        <!-- Complex Case -->
        <path d="M0,0 L150,0 L150,100 L100,100 L100,50 L0,50 Z" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
        <rect x="20" y="10" width="40" height="30" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <rect x="110" y="60" width="60" height="50" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Complex Case</text>
        <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">Mixed relationships possible</text>
        <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">Precise geometry handling</text>
    </g>

    <g transform="translate(240,280)">
        <!-- No Relation - SP element moved further to avoid overlap -->
        <rect x="0" y="0" width="150" height="100" fill="#E1F0E5" stroke="#2E7D32" stroke-width="2"/>
        <rect x="170" y="20" width="60" height="40" fill="#FFF3E0" stroke="#E65100" stroke-width="2"/>
        <text x="75" y="130" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">No Relation</text>
        <text x="75" y="150" text-anchor="middle" font-family="Arial" font-size="12">sp_relation_type = "no_relation"</text>
        <text x="75" y="165" text-anchor="middle" font-family="Arial" font-size="12">has_sp_elements = False</text>
    </g>

    <!-- Processing Features Box - Moved further right -->
    <g transform="translate(520,280)">
        <rect x="0" y="0" width="260" height="120" fill="#F5F5F5" stroke="#666666" stroke-width="1"/>
        <text x="130" y="25" text-anchor="middle" font-family="Arial" font-size="14" font-weight="bold">Processing Features</text>
        <text x="10" y="50" font-family="Arial" font-size="12">• Parallel processing for performance</text>
        <text x="10" y="70" font-family="Arial" font-size="12">• Spatial indexing for efficient filtering</text>
        <text x="10" y="90" font-family="Arial" font-size="12">• Precise geometric operations</text>
        <text x="10" y="110" font-family="Arial" font-size="12">• Handles invalid geometries automatically</text>
    </g>

    <!-- Legend Box - Simplified -->
    <g transform="translate(40,460)">
        <!-- Background -->
        <rect x="-10" y="-10" width="500" height="60" fill="white" stroke="#666666" stroke-width="1"/>
        
        <!-- Legend content -->
        <g transform="translate(20,20)">
            <!-- First item -->
            <g transform="translate(0,0)">
                <rect x="0" y="-10" width="20" height="20" fill="#E1F0E5" stroke="#2E7D32" stroke-width="1"/>
                <text x="30" y="5" font-family="Arial" font-size="14">Roof polygon (gdf_toiture)</text>
            </g>
            
            <!-- Second item -->
            <g transform="translate(250,0)">
                <rect x="0" y="-10" width="20" height="20" fill="#FFF3E0" stroke="#E65100" stroke-width="1"/>
                <text x="30" y="5" font-family="Arial" font-size="14">SP element (gdf_toiture_sp)</text>
            </g>
        </g>
    </g>
</svg>"""

# Sauvegarde et affichage du SVG
save_and_display_svg(
    gdf_toiture_6_contains_SP_svg_content, 
    f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_05_gdf_toiture_6_contains_SP.svg"
)

In [ ]:
def analyze_sp_elements_presence(gdf_toiture, gdf_toiture_sp):
    """
    Analyse la présence et les relations spatiales des éléments SP 
    (superstructures) dans les polygones de toiture.
    
    Cette fonction effectue une analyse spatiale détaillée pour déterminer 
    comment les éléments SP interagissent avec les polygones de toiture.
    
    Parameters:
        gdf_toiture (GeoDataFrame): Polygones de toiture
        gdf_toiture_sp (GeoDataFrame): Éléments SP
    
    Returns:
        GeoDataFrame: GeoDataFrame enrichi avec les informations SP
    """
    # Suppression des avertissements non critiques
    warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    # Initialisation et préparation des données
    gdf_result = gdf_toiture.copy()

    # Alignement des systèmes de coordonnées
    if gdf_toiture.crs != gdf_toiture_sp.crs:
        print("Conversion du CRS pour correspondre aux polygones de toiture...")
        gdf_toiture_sp = gdf_toiture_sp.to_crs(gdf_toiture.crs)

    # Validation des géométries
    print("Validation des géométries...")
    gdf_result["geometry"] = gdf_result.geometry.make_valid()
    gdf_toiture_sp["geometry"] = gdf_toiture_sp.geometry.make_valid()

    # Initialisation des colonnes de résultats
    gdf_result["sp_contains"] = False
    gdf_result["sp_within"] = False
    gdf_result["sp_overlaps"] = False
    gdf_result["sp_element_count"] = 0
    gdf_result["sp_relation_type"] = "no_relation"

    # Création de l'index spatial pour optimisation
    spatial_index = gdf_toiture_sp.sindex

    def process_roof(roof_idx, roof_geom, sp_data, spatial_index):
        """
        Traite un polygone de toiture contre tous les éléments SP pertinents.
        
        Parameters:
            roof_idx: Index du toit
            roof_geom: Géométrie du toit
            sp_data: Données SP
            spatial_index: Index spatial
            
        Returns:
            dict: Résultats de l'analyse spatiale
        """
        try:
            # Filtrage spatial initial
            possible_matches_idx = list(spatial_index.intersection(roof_geom.bounds))

            if not possible_matches_idx:
                return {
                    'idx': roof_idx,
                    'contains': False,
                    'within': False,
                    'overlaps': False,
                    'count': 0,
                    'relation': "no_relation"
                }

            # Analyse spatiale raffinée
            possible_matches = sp_data.iloc[possible_matches_idx]
            intersecting = possible_matches[possible_matches.geometry.intersects(roof_geom)]

            if intersecting.empty:
                return {
                    'idx': roof_idx,
                    'contains': False,
                    'within': False,
                    'overlaps': False,
                    'count': 0,
                    'relation': "no_relation"
                }

            # Analyse géométrique détaillée
            try:
                clipped = intersecting.copy()
                clipped['clipped_geom'] = intersecting.geometry.intersection(roof_geom)
                clipped = clipped[~clipped['clipped_geom'].is_empty]

                if clipped.empty:
                    return {
                        'idx': roof_idx,
                        'contains': False,
                        'within': False,
                        'overlaps': False,
                        'count': 0,
                        'relation': "no_relation"
                    }

                # Détermination des relations spatiales précises
                contains_any = any(roof_geom.contains(geom) for geom in clipped['geometry'])
                within_any = any(roof_geom.within(geom) for geom in clipped['geometry'])

                relation_type = "overlaps"
                if contains_any:
                    relation_type = "contains"
                elif within_any:
                    relation_type = "within"

                return {
                    'idx': roof_idx,
                    'contains': contains_any,
                    'within': within_any,
                    'overlaps': not (contains_any or within_any),
                    'count': len(clipped),
                    'relation': relation_type
                }

            except Exception as e:
                print(f"Attention: Échec du clipping pour le toit {roof_idx}: {e}")
                return {
                    'idx': roof_idx,
                    'contains': False,
                    'within': False,
                    'overlaps': False,
                    'count': 0,
                    'relation': "error"
                }

        except Exception as e:
            print(f"Attention: Erreur lors du traitement du toit à l'index {roof_idx}: {e}")
            return {
                'idx': roof_idx,
                'contains': False,
                'within': False,
                'overlaps': False,
                'count': 0,
                'relation': "error"
            }

    # Configuration du traitement parallèle
    n_cores = multiprocessing.cpu_count()
    batch_size = max(1, len(gdf_result) // (n_cores * 2))
    n_workers = min(n_cores, max(1, len(gdf_result) // batch_size))

    print(f"\nDémarrage du traitement parallèle avec {n_workers} workers")
    print(f"Taille des lots: {batch_size}")

    # Exécution du traitement parallèle
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = []
        for idx, roof in gdf_result.iterrows():
            futures.append(
                executor.submit(
                    process_roof,
                    idx,
                    roof.geometry,
                    gdf_toiture_sp,
                    spatial_index
                )
            )

        # Collecte des résultats avec barre de progression
        results = []
        for future in tqdm(as_completed(futures), 
                          total=len(futures), 
                          desc="Traitement des toitures"):
            results.append(future.result())

    # Mise à jour des résultats
    print("\nMise à jour des résultats...")
    for result in results:
        idx = result['idx']
        gdf_result.at[idx, "sp_contains"] = result['contains']
        gdf_result.at[idx, "sp_within"] = result['within']
        gdf_result.at[idx, "sp_overlaps"] = result['overlaps']
        gdf_result.at[idx, "sp_element_count"] = result['count']
        gdf_result.at[idx, "sp_relation_type"] = result['relation']

    # Calcul du flag de présence globale SP
    gdf_result["has_sp_elements"] = (
        gdf_result["sp_contains"] | 
        gdf_result["sp_within"] | 
        gdf_result["sp_overlaps"]
    )

    # Statistiques récapitulatives
    print("\nAnalyse terminée - Statistiques récapitulatives:")
    print(f"Total de polygones de toiture traités: {len(gdf_result)}")
    print("\nDistribution des relations spatiales:")
    print(gdf_result["sp_relation_type"].value_counts())
    print("\nDistribution du nombre d'éléments SP:")
    print(gdf_result["sp_element_count"].value_counts().sort_index())

    return gdf_result

# Exécution de l'analyse spatiale détaillée
gdf_toiture_6_contains_SP = analyze_sp_elements_presence(
    gdf_toiture_5_hole, gdf_toiture_sp_2_filtre_egid
)

# Vérification des résultats
print("\nRésultats de l'analyse:")
print(
    f"Total de toitures avec éléments SP: {gdf_toiture_6_contains_SP['has_sp_elements'].sum()}"
)
print("\nTypes de relations:")
print(gdf_toiture_6_contains_SP["sp_relation_type"].value_counts())


In [ ]:
def create_report_plots(df):
    """
    Crée une série de graphiques de qualité publication pour le rapport d'analyse spatiale.
    """
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_palette("husl")

    # 1. Graphique de distribution des relations
    plt.figure(figsize=(12, 8), dpi=300)
    relationship_counts = df['sp_relation_type'].value_counts()

    # Graphique en barres avec couleurs personnalisées
    bars = sns.barplot(x=relationship_counts.index, 
                      y=relationship_counts.values,
                      palette=['#3498db', '#2ecc71', '#e74c3c', '#f1c40f'])

    # Ajout des valeurs et pourcentages
    total = len(df)
    for i, v in enumerate(relationship_counts.values):
        percentage = v/total*100
        plt.text(i, v, f'{v}', 
                ha='center', va='bottom', fontsize=10)
        plt.text(i, v/2, f'{percentage:.1f}%',
                ha='center', va='bottom', 
                color='white', fontweight='bold', fontsize=10)

    plt.title('Distribution des relations spatiales\nentre toitures et éléments SP',
              fontsize=16, pad=20, fontweight='bold')
    plt.xlabel('Type de relation', fontsize=12, labelpad=10)
    plt.ylabel('Nombre de toitures (échelle log)', fontsize=12, labelpad=10)
    plt.yscale('log')
    plt.xticks(rotation=30)

    # Boîte de description
    description = (
        f'Total toitures: {total}\n'
        f'Avec éléments SP: {df["has_sp_elements"].sum()} ({df["has_sp_elements"].mean()*100:.1f}%)'
    )
    plt.text(0.95, 0.95, description,
             transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8),
             ha='right', va='top', fontsize=10)

    plt.savefig(
        f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_06_analyse_relation_toitures_avec_sp_distribution.png",
        dpi=300,
        bbox_inches="tight",
    )

    plt.tight_layout()
    plt.show()

    # 2. Distribution du nombre d'éléments (excluant no_relation)
    plt.figure(figsize=(15, 8), dpi=300)

    df_with_sp = df[df['sp_relation_type'] != 'no_relation']

    sns.violinplot(data=df_with_sp, x='sp_relation_type', y='sp_element_count',
                  palette=['#3498db', '#e74c3c', '#f1c40f'])

    plt.title('Distribution du nombre d\'éléments SP par type de relation',
              fontsize=16, pad=20, fontweight='bold')
    plt.xlabel('Type de relation', fontsize=12, labelpad=10)
    plt.ylabel('Nombre d\'éléments SP', fontsize=12, labelpad=10)

    # Statistiques récapitulatives
    stats = df_with_sp.groupby('sp_relation_type')['sp_element_count'].agg([
        'mean', 'median', 'std', 'max'
    ]).round(2)

    stats_text = "Statistiques récapitulatives:\n"
    for idx in stats.index:
        stats_text += f"\n{idx}:\n"
        stats_text += f"  Moyenne: {stats.loc[idx, 'mean']}\n"
        stats_text += f"  Médiane: {stats.loc[idx, 'median']}\n"
        stats_text += f"  Écart-type: {stats.loc[idx, 'std']}\n"
        stats_text += f"  Maximum: {stats.loc[idx, 'max']}\n"

    plt.text(1.15, 0.5, stats_text,
             transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8),
             fontsize=10, va='center')

    plt.savefig(
        f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_08_analyse_relation_toitures_avec_sp_element_count.png",
        dpi=300,
        bbox_inches="tight",
    )

    plt.tight_layout()
    plt.show()

    # 3. Histogramme détaillé du nombre d'éléments SP
    plt.figure(figsize=(15, 8), dpi=300)

    relationship_types = ['contains', 'overlaps', 'within']
    colors = ['#3498db', '#e74c3c', '#f1c40f']

    for rtype, color in zip(relationship_types, colors):
        data = df_with_sp[df_with_sp['sp_relation_type'] == rtype]['sp_element_count']
        plt.hist(data, bins=50, alpha=0.5, label=rtype, color=color)

    plt.title('Histogramme du nombre d\'éléments SP par type de relation',
              fontsize=16, pad=20, fontweight='bold')
    plt.xlabel('Nombre d\'éléments SP', fontsize=12, labelpad=10)
    plt.ylabel('Fréquence (échelle log)', fontsize=12, labelpad=10)
    plt.yscale('log')
    plt.legend(title='Type de relation', fontsize=10, title_fontsize=12)

    # Texte récapitulatif
    summary_text = "Caractéristiques de distribution:\n"
    for rtype in relationship_types:
        data = df_with_sp[df_with_sp['sp_relation_type'] == rtype]['sp_element_count']
        summary_text += f"\n{rtype}:\n"
        summary_text += f"  Nombre: {len(data)}\n"
        summary_text += f"  Moyenne: {data.mean():.2f}\n"
        summary_text += f"  Médiane: {data.median():.2f}\n"
        summary_text += f"  95e percentile: {data.quantile(0.95):.2f}\n"

    plt.text(1.15, 0.5, summary_text,
             transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8),
             fontsize=10, va='center')

    plt.savefig(
        f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_08_analyse_relation_toitures_avec_sp_histogramme.png",
        dpi=300,
        bbox_inches="tight",
    )

    plt.tight_layout()
    plt.show()

# Création et affichage des graphiques
create_report_plots(gdf_toiture_6_contains_SP)

In [ ]:
display(gdf_toiture_6_contains_SP.columns)
display(gdf_toiture_6_contains_SP.head(2))
display(gdf_toiture_6_contains_SP.dtypes)

In [ ]:
# Sauvegarde des résultats
gdf_toiture_6_contains_SP.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_6_contains_SP.parquet"
)

### Application de la classification

In [ ]:
gdf_toiture_7_classification = gdf_toiture_6_contains_SP.copy()

# Initialisation des colonnes de classification
gdf_toiture_7_classification["classification"] = None
gdf_toiture_7_classification["classification_simplified"] = None
gdf_toiture_7_classification["classification_comment"] = None

# Classe 1: Toiture totalement occupée (≤ 2m²)
mask_classe1 = (
    ~gdf_toiture_7_classification["shape_area_sup_2m2"]
    )
gdf_toiture_7_classification.loc[
    mask_classe1,
    ["classification", "classification_simplified" , "classification_comment"],
] = [
    "1",
    "1",
    "Classe 1: Toiture totalement occupée"]

# Classe 2a: Toiture partiellement occupée sans trou
mask_classe2a = (
    (gdf_toiture_7_classification["shape_area_sup_2m2"]) &
    (~gdf_toiture_7_classification["has_fully_contained_holes"]) &
    (gdf_toiture_7_classification["sp_element_count"] >= 1)
)
gdf_toiture_7_classification.loc[
    mask_classe2a,
    ["classification", "classification_simplified", "classification_comment"]] = [
    "2a",
    "2",
    "Classe 2a: Toiture partiellement occupée sans trou",
]

# Classe 2b: Toiture partiellement occupée avec trou
mask_classe2b = (
    (gdf_toiture_7_classification["shape_area_sup_2m2"]) &
    (gdf_toiture_7_classification["has_fully_contained_holes"]) &
    (gdf_toiture_7_classification["sp_element_count"] >= 1)
)
gdf_toiture_7_classification.loc[
    mask_classe2b,
    ["classification", "classification_simplified", "classification_comment"],
] = [
    "2b",
    "2",
    "Classe 2b: Toiture partiellement occupée avec trou"]

# Classe 3a: Toiture libre sans trou
mask_classe3a = (
    (gdf_toiture_7_classification["shape_area_sup_2m2"]) &
    (~gdf_toiture_7_classification["has_fully_contained_holes"]) &
    (gdf_toiture_7_classification["sp_element_count"] == 0)
)
gdf_toiture_7_classification.loc[
    mask_classe3a,
    ["classification", "classification_simplified", "classification_comment"],
] = ["3a", "3", "Classe 3a: Toiture libre sans trou"]

# Classe 3b: Toiture libre avec trou
mask_classe3b = (
    (gdf_toiture_7_classification["shape_area_sup_2m2"]) &
    (gdf_toiture_7_classification["has_fully_contained_holes"]) &
    (gdf_toiture_7_classification["sp_element_count"] == 0)
)
gdf_toiture_7_classification.loc[
    mask_classe3b,
    ["classification", "classification_simplified", "classification_comment"],
] = ["3b", "3" ,"Classe 3b: Toiture libre avec trou"]

# Résumé de la classification
print("\nDistribution de la classification:")
print(gdf_toiture_7_classification["classification"].value_counts())

# Vérification des toitures non classifiées
unclassified = gdf_toiture_7_classification[gdf_toiture_7_classification["classification"].isna()]
if not unclassified.empty:
    print("\nAttention: Certaines toitures n'ont pas été classifiées!")
    print(f"Nombre de toitures non classifiées: {len(unclassified)}")
    print("\nÉchantillon de toitures non classifiées:")
    print("Propriétés des toitures non classifiées:")
    print(unclassified[["shape_area_sup_2m2", "has_fully_contained_holes", "sp_element_count"]].head())

# Statistiques détaillées
print("\nStatistiques détaillées de classification:")
for classification in sorted(gdf_toiture_7_classification["classification"].unique()):
    if classification is not None:
        subset = gdf_toiture_7_classification[gdf_toiture_7_classification["classification"] == classification]
        print(f"\nClasse {classification}:")
        print(f"Nombre: {len(subset)}")
        print("Propriétés moyennes:")
        print(f"Nombre d'éléments SP: {subset['sp_element_count'].mean():.2f}")

# Sauvegarde des résultats
gdf_toiture_7_classification.to_parquet(
   f"{OUTPUT_PARQUET_NOTEBOOK_02_PATH}/02_gdf_toiture_7_classification.parquet"
)


### Visualisation des résultats

In [ ]:
# Définition des couleurs pour chaque classe
class_colors = {
    "1": "red",
    "2a": "#d95f02",
    "2b": "#fdcdac",
    "3a": "green",
    "3b": "#66a61e",
}

# Création de la figure
plt.figure(figsize=(10, 6))

# Graphique en barres avec style personnalisé
ax = sns.countplot(
    data=gdf_toiture_7_classification,
    x="classification",
    order=sorted(gdf_toiture_7_classification["classification"].unique()),
    hue="classification",
    palette=class_colors,
    legend=False
)

# Personnalisation du graphique
plt.title(f"Distribution des classifications de toitures ({len(gdf_toiture_7_classification)} toitures)", fontsize=14)
plt.xlabel("Classification des toitures", fontsize=12)
plt.ylabel("Nombre de toitures", fontsize=12)

# Grille pour améliorer la lisibilité
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Ajout des valeurs sur les barres
for p in ax.patches:
    height = p.get_height()
    ax.annotate(
        f"{int(height)}",
        xy=(p.get_x() + p.get_width() / 2, height),
        textcoords="offset points",
        xytext=(0, 3),
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="bold"
    )

# Descriptions des classes
class_descriptions = {
    "1": "Totalement occupée",
    "2a": "Partiellement occupée\nsans trou",
    "2b": "Partiellement occupée\navec trou",
    "3a": "Libre sans trou",
    "3b": "Libre avec trou"
}

# Configuration de l'axe X avec descriptions
plt.xticks(range(len(class_descriptions)), 
           [f"Classe {k}\n{v}" for k, v in class_descriptions.items()],
           rotation=0,
           ha='center')

plt.tight_layout()

# Sauvegarde
plt.savefig(
    f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_09_gdf_toiture_7_classification_01_complete.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)

plt.show()

# Distribution en pourcentage
total = len(gdf_toiture_7_classification)
print("\nDistribution des classifications (pourcentage):")
for classe in sorted(gdf_toiture_7_classification["classification"].unique()):
    count = len(gdf_toiture_7_classification[gdf_toiture_7_classification["classification"] == classe])
    percentage = (count / total) * 100
    print(f"Classe {classe}: {percentage:.1f}% ({count} toitures)")

In [ ]:
# Graphique pour la classification simplifiée
class_colors = {
    "1": "red",
    "2": "orange",
    "3": "green",
}

plt.figure(figsize=(10, 6))

ax = sns.countplot(
    data=gdf_toiture_7_classification,
    x="classification_simplified",
    order=sorted(gdf_toiture_7_classification["classification_simplified"].unique()),
    hue="classification_simplified",
    palette=class_colors,
    legend=False,
)

plt.title(
    f"Distribution des classifications simplifiées de toitures ({len(gdf_toiture_7_classification)} toitures)",
    fontsize=14,
)
plt.xlabel("Classification simplifiée des toitures", fontsize=12)
plt.ylabel("Nombre de toitures", fontsize=12)

plt.grid(axis="y", linestyle="--", alpha=0.7)

# Ajout des valeurs sur les barres
for p in ax.patches:
    height = p.get_height()
    ax.annotate(
        f"{int(height)}",
        xy=(p.get_x() + p.get_width() / 2, height),
        textcoords="offset points",
        xytext=(0, 3),
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="bold",
    )

# Descriptions des classes simplifiées
class_descriptions = {
    "1": "Totalement occupée",
    "2": "Partiellement occupée",
    "3": "Libre",
}

plt.xticks(
    range(len(class_descriptions)),
    [f"Classe {k}\n{v}" for k, v in class_descriptions.items()],
    rotation=0,
    ha="center",
)

plt.tight_layout()

plt.savefig(
    f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_10_gdf_toiture_7_classification_02_simplified.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)

plt.show()

# Distribution en pourcentage
total = len(gdf_toiture_7_classification)
print("\nDistribution des classifications simplifiées (pourcentage):")
for classe in sorted(
    gdf_toiture_7_classification["classification_simplified"].unique()
):
    count = len(
        gdf_toiture_7_classification[
            gdf_toiture_7_classification["classification_simplified"] == classe
        ]
    )
    percentage = (count / total) * 100
    print(f"Classe {classe}: {percentage:.1f}% ({count} toitures)")


In [ ]:
# Analyse par catégorie SIA
# Gestion des valeurs manquantes
gdf_toiture_7_classification['sia_cat'] = gdf_toiture_7_classification['sia_cat'].fillna('Non défini').astype(str)

# Définition des couleurs
class_colors = {
    "1": "red",  
    "2a": "#d95f02",  
    "2b": "#fdcdac",  
    "3a": "green",
    "3b": "#66a61e",
}

# Descriptions des classes
class_descriptions = {
    "1": "Totalement\noccupée",
    "2a": "Partiellement\noccupée\nsans trou",
    "2b": "Partiellement\noccupée\navec trou",
    "3a": "Libre\nsans trou",
    "3b": "Libre\navec trou"
}

# Catégories SIA uniques
sia_cats = sorted(gdf_toiture_7_classification['sia_cat'].unique())
n_cats = len(sia_cats)

# Dimensions de la grille de sous-graphiques
n_cols = 3
n_rows = int(np.ceil(n_cats / n_cols))

# Création de la figure
fig = plt.figure(figsize=(20, 25))

# Titre principal
total_roofs = len(gdf_toiture_7_classification)
fig.suptitle('Distribution des classifications de toitures par catégorie SIA\n' + 
             f'Total: {total_roofs} toitures',
             fontsize=16, y=0.95)

# Noms personnalisés des catégories
category_names = {
    "A": "Habitat collectif",
    "B": "Habitat individuel",
    "C": "Administration",
    "D": "Écoles",
    "E": "Industrie",
    "F": "Non défini",
    "G": "Commerce",
    "H": "Restauration",
    "I": "Lieux de rassemblement",
    "K": "Hôtellerie",
    "L": "Dépots",
    "M": "Installations sportives",
    "N": "Piscines couvertes"
}

# Création des sous-graphiques pour chaque catégorie SIA
for idx, cat in enumerate(sia_cats):
    # Données pour cette catégorie
    cat_data = gdf_toiture_7_classification[gdf_toiture_7_classification['sia_cat'] == cat]
    total_cat = len(cat_data)
    
    # Création du sous-graphique
    ax = plt.subplot(n_rows, n_cols, idx + 1)
    
    # Calcul des comptages et pourcentages
    counts = cat_data['classification'].value_counts().reindex(sorted(class_colors.keys())).fillna(0)
    
    # Création des barres
    bars = ax.bar(range(len(counts)), counts, color=[class_colors[x] for x in counts.index])
    
    # Nom de la catégorie
    category_name = category_names.get(cat, cat)
    
    # Titre et labels
    ax.set_title(f'Catégorie SIA {category_name}\n{total_cat} toitures', 
                 fontsize=12, pad=10)
    ax.set_ylabel("Nombre de toitures", fontsize=10)
    
    # Ajout des valeurs sur les barres
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            pct = (height / total_cat) * 100
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(height)}\n({pct:.1f}%)',
                   ha='center', va='bottom', fontsize=9)
    
    # Configuration de l'axe X
    ax.set_xticks(range(len(class_descriptions)))
    ax.set_xticklabels([f"Classe {k}\n{v}" for k, v in class_descriptions.items()], 
                       fontsize=8, rotation=0)
    
    # Grille
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    
    # Ajustement de l'axe Y pour les labels
    ax.set_ylim(0, ax.get_ylim()[1] * 1.1)

# Ajustement de la mise en page
plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.savefig(
    f"{OUTPUT_GRAPHICS_NOTEBOOK_02_PATH}/02_11_gdf_toiture_7_classification_03_by_sia_separate.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)

plt.show()

# Statistiques détaillées par catégorie SIA
print("\nStatistiques détaillées par catégorie SIA:")
for cat in sia_cats:
    cat_data = gdf_toiture_7_classification[gdf_toiture_7_classification['sia_cat'] == cat]
    total_cat = len(cat_data)
    category_name = category_names.get(cat, cat)
    print(f"\n{category_name} (Total: {total_cat} toitures):")
    
    class_counts = cat_data['classification'].value_counts().sort_index()
    for classe, count in class_counts.items():
        pct = (count / total_cat) * 100
        print(f"  Classe {classe}: {pct:.1f}% ({count} toitures)")

# Tableau récapitulatif
summary_table = pd.pivot_table(
    gdf_toiture_7_classification,
    index='sia_cat',
    columns='classification',
    aggfunc='size',
    fill_value=0
)

# Ajout des colonnes de pourcentage
for col in summary_table.columns:
    summary_table[f'{col}_pct'] = (summary_table[col] / summary_table.sum(axis=1) * 100).round(1)

# Remplacement de l'index par les noms de catégories
summary_table.index = summary_table.index.map(category_names)

print("\nTableau récapitulatif (nombres et pourcentages):")
print(summary_table)